In [11]:
import pandas as pd 
import geopandas as gp
from shapely.ops import nearest_points
from shapely.geometry import Point

In [14]:
path = r'/home/semipro321/Documents/Workspace/Geospatial-Analysis-on-House-Price-Prediction/Inputs/'

In [15]:
df_house = pd.read_csv(path + 'House/house_with_neigh.csv')
df_school = pd.read_csv(path + 'City/Schools_with_neigh.csv')
gpf_house = gp.GeoDataFrame(df_house, crs="EPSG:4326", geometry=gp.points_from_xy(df_house.long,df_house.lat))
gpf_school= gp.GeoDataFrame(df_school, crs="EPSG:4326", geometry=gp.points_from_xy(df_school.LONGITUDE,df_school.LATITUDE))


In [4]:
def near(df, col: [], point, pts):
     '''
     df := pass in the df you want to search
     col := the column you want to return (LISt)
     point := the geometry of left dataset
     pts := union of dataset
     '''
     nearest = df.geometry == nearest_points(point, pts)[1]
     return df[nearest][col].values[0]


nearestschools = gpf_house.apply(lambda row: near(gpf_school, ['_id', 'NAME'], row.geometry, gpf_school.geometry.unary_union), axis=1)

In [5]:
nearestschools = nearestschools.apply(pd.Series)
nearestschools.columns = ['_id', 'NAME']
mergegp = gpf_school.merge(nearestschools, on='_id', how='right')

In [6]:
def calcdistance(origindf, destdf):
    origindf = origindf['geometry'].to_crs('EPSG:5234')
    destdf = destdf['geometry'].to_crs('EPSG:5234')
    return origindf.distance(destdf)
gpf_house['distance_nearest_school'] = calcdistance(gpf_house, mergegp)

In [7]:
df_ttc = pd.read_csv(path + 'Stops.csv')
gpf_ttc = gp.GeoDataFrame(df_ttc, crs="EPSG:4326", geometry=gp.points_from_xy(df_ttc.stop_lon,df_ttc.stop_lat))


In [8]:
nearestttc = gpf_house.apply(lambda row: near(gpf_ttc, ['stop_id'], row.geometry, gpf_ttc.geometry.unary_union), axis=1)
nearestttc = nearestttc.apply(pd.Series)
nearestttc.columns = ['stop_id']
mergegpttc = gpf_ttc.merge(nearestttc, on='stop_id', how='right')

In [9]:
gpf_house.distance_nearest_school

0        20079.752292
1        18799.767218
2        18799.767218
3        18742.153658
4        18742.153658
             ...     
15229    20821.689106
15230    13885.715038
15231    13654.574954
15232    13631.505174
15233    13855.710760
Name: distance_nearest_school, Length: 15234, dtype: float64

ValueError: Must provide an even number of non-keyword arguments